In [ ]:
import os
import sys
import csv
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

sys.path.append('../')
from envir import config


# #### connect & load data

engine = create_engine('postgresql+psycopg2:///yelp_abcs')
con = engine.connect

yelp = pd.read_sql("select * from stag.yelp_academic_business where state in ('ON', 'NV');", con=engine )
vegas = pd.read_sql("select * from clean.clean_vegas_violations;", con=engine)
toronto = pd.read_sql("select * from clean.toronto_inspections_cleaned;", con=engine)

yelp2 = yelp.loc[yelp['categories'].str.contains("Restaurant")==True][['business_id',
                                                                       'name', 'address', 'city', 'state',
                                                                       'postal_code', 'latitude', 'longitude']].copy()

yelp2.reset_index(inplace=True)

len(toronto) + len(vegas)
#toronto.drop(columns=['Unnamed: 0'], inplace=True)
vegas.drop(columns=['index'], inplace=True)

violations = vegas.append(toronto, ignore_index=True, sort=True)

# #### get unique business names from violations
biz = violations[['name', 'address', 'city']].drop_duplicates().copy()
biz['id'] = biz.index
biz.reset_index(inplace=True)

# these will be the match choices
name_choices = biz['name'] + ' ' +biz['address']

# these are the ones to find
name = yelp2['name'] + ' ' + yelp2['address']

with open(config.shared+'fuzzy_biz_names75.csv', mode='w') as csv_file:
    fieldnames = ['yelp_name_address', 'violations_name_address', 'match_score']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    
    for i in name:
        matched = process.extractOne(i, name_choices, scorer=fuzz.token_sort_ratio, score_cutoff=75)
        if matched is not None:
            writer.writerow({'yelp_name_address': i,
                             'violations_name_address': matched[0],
                             'match_score': matched[1]})
        writer.writerow({'yelp_name_address': i,
                         'violations_name_address': None,
                         'match_score': None})